# SADO Expert System

In [1]:
from experta import *

### Facts

In [40]:
class Age(Fact):
    pass

class Height(Fact):
    pass

class Weight(Fact):
    pass

class Gender(Fact):
    pass

class Calories(Fact):
    pass

class Competing(Fact):
    pass

class Intensity(Fact):
    pass

class BMI(Fact):
    pass

class DietPlan(Fact):
    pass

class setOption(Fact):
    pass
    
class Option(Fact):
    foodSet = Field(str, default="")
    option = Field(str, default="")
    calories = Field(str, default="")
    
class setSet(Fact):
    pass
    
class Set(Fact):
    setCategory = Field(str, default="")
    setFoods = Field(list, default=[])
    setCalories = Field(str, default="")
    
class Ask(Fact):
    pass

## DefFacts & Rules

In [75]:
class SADO(KnowledgeEngine):
    @DefFacts()
    def sado_rules(self):
        
        ''' set facts for all food set '''       
        yield Option(foodSet="casual",option="A",calories="<1500") # maybe tak perlu simpan calories
        yield Option(foodSet="casual",option="B",calories=">=1500 & <2500")
        yield Option(foodSet="casual",option="C",calories=">=2500")
        yield Option(foodSet="moderate",option="D",calories="<2500")
        yield Option(foodSet="moderate",option="E",calories=">=2500 & <3500")
        yield Option(foodSet="moderate",option="F",calories=">=3500 & <4500")
        
        #yield Option(foodSet="moderate",option=["D","E","F"],calories=["<2500",">=2500 & <3500",">=3500 & <4500"])
        #yield Option(foodSet="intensive",option=["G","H","I"],calories=["<3500",">=3500 & <4500",">=4500 & <5500"])
        
        yield Set(setCategory="D",foodsRec=["Nasi Lemak","Nasi xde Lemak","Wahhh banyak lemak kau"],setCalories="<2500")
        yield Set(setCategory="E",foodsRec=["Nasi Goreng","Nasi x Goreng","Goreng le sendiri"],
                  setCalories=">=2500 & <3500")
        yield Set(setCategory="F",foodsRec=["Nasi Mamak","Nasi xde Mamak","Teh ais sune"],
                  setCalories=">=3500 & <4500")
        #yield Set(setCategory="e",foodsRec=["Nasi Goreng","Nasi x Goreng","Goreng le sendiri"],
        #          setCalories=">=2500 & >3500")
    
    @Rule()
    def startup(self):
        ''' input age, height, weight and gender for calories and bmi '''
        print("Welcome to SADO system")
        questionAge = "Please enter your age: "
        resAge = input(questionAge).lower()
        questionHeight = "Please enter your height (in cm): "
        resHeight = input(questionHeight).lower()
        questionWeight = "Please enter your weight (in kg): "
        resWeight = input(questionWeight).lower()
        questionGender = "Please enter your gender: "
        resGender = input(questionGender).lower()
        self.declare(Age(resAge))
        self.declare(Height(resHeight))
        self.declare(Weight(resWeight))
        self.declare(Gender(resGender))
        
    @Rule(AS.f1 << Gender(MATCH.g),
          AS.f2 << Age(MATCH.a),
          AS.f3 << Height(MATCH.h),
          AS.f4 << Weight(MATCH.w))
    def gender(self, g, a, h, w):
        ''' if male/female, count calories with formula for each gender, then ask if competing '''
        if(g == "male"):
            self.declare(Calories((10*int(w))+(6.25*int(h))-(5*int(a))+5))
            print("You are a " + g)
        elif(g =="female"):
            self.declare(Calories((10*int(w))+(6.25*int(h))-(5*int(a))+161))
            print("You are a " + g)
        else:
            print("Invalid gender lul")
        self.declare(Ask("competition-participation")) # todo - sepatutnya takde if else dalam function, buat rules baru
            
    @Rule(AS.f1 << Ask("competition-participation"))
    def if_competing(self, f1):
        ''' determine if competing or not '''
        resCompeting = input("Are you participating in a competition? [y/n] : ").lower()
        self.declare(Competing(resCompeting))
        #self.retract(f1)  Watch out watch out!
        if(resCompeting == 'y'):
            print("You are competing!")
        elif(resCompeting == 'n'):
            print("You are not competing!")
            
    ''' Rule Section - Determine if competition or not '''
            
    @Rule(AS.f1 << Competing('n'))
    def not_competing(self, f1):
        ''' if not competing, recommend casual planning '''
        self.declare(DietPlan("casual"))
        
    @Rule(AS.f1 << Competing('y'),
          AS.f2 << Height(MATCH.h),
          AS.f3 << Weight(MATCH.w))
    def is_competing(self, h, w):
        print("How intense are your training?")
        resIntensity = input("Choose one of the options [low/medium/high]: ").lower()
        heightInM = int(h)/100
        bmi = int(w)/pow(heightInM,2)
        self.declare(Intensity(resIntensity))
        print("You are working out at a " + resIntensity + " intensity")
        if(bmi>30):
           self.declare(BMI("overweight")) # ok you know what, maybe in some condition ada if dalam rule
        
    ''' Rules Section - Determine diet plan for each groups of rule if in competition '''
        
    @Rule(AS.f1 << Intensity("low"),
          OR(
              BMI("normal"),
              BMI("overweight")
             )
          )
    def low_normal_overweight(self):
        self.declare(DietPlan("moderate")) # change moderate to low_intensive or other name related
        print("For a moderate diet plan, these are the options")
        # set Option(calories=Calories())
        # A if Calories() in range and else ...
        
    @Rule(AS.f1 << DietPlan("moderate"),
          Option(foodSet = "moderate",
                 option = MATCH.o,
                 calories = MATCH.c))
    def display_moderate_plans(self, o, c):
        # display semua set dulu
        print(o + "(" + c + "kcal)")
            
    @Rule(AS.f1 << DietPlan("moderate"),
          AS.f2 << Calories(MATCH.cc),
          Option(foodSet = "moderate",
                 option = MATCH.o,
                 calories = MATCH.c))
    def set_calorie_range(self, cc):
        # maybe kena buat rule baru for these
        if (cc < 2500):
            self.declare(setOption("D"))
        elif (cc >= 2500 & cc < 3500):
            self.declare(setOption("E"))
        elif (cc >=3500 & cc <4500):
            self.declare(setOption("F"))
        
    ''' Rules Section - Show options for each diet plan '''
    
    # todo - dah ada calorie target then display set yang recommended        
     
    @Rule(setOption(MATCH.o),
          Calories(MATCH.cc),
          Set(setCategory = MATCH.o,
              foodsRec = MATCH.f,
              setCalories = MATCH.c))
    def display_foods(self, o, cc, f, c):
        print("Considering you need " , cc , "kcal, you are recommended eat these in set " + o +": ")
        for z in f:
            print(z + "\t", end="")
        print()
        print("All foods in this set contains calories in the " + c + " range.")

In [76]:
sado = SADO()
sado.reset()
sado.run()

Welcome to SADO system
Please enter your age: 20
Please enter your height (in cm): 164
Please enter your weight (in kg): 90
Please enter your gender: male
You are a male
Are you participating in a competition? [y/n] : y
You are competing!
How intense are your training?
Choose one of the options [low/medium/high]: low
You are working out at a low intensity
For a moderate diet plan, these are the options
Considering you need  1830.0 kcal, you are recommended eat these in set D: 
Nasi Lemak	Nasi xde Lemak	Wahhh banyak lemak kau	
All foods in this set contains calories in the <2500 range.
F(>=3500 & <4500kcal)
E(>=2500 & <3500kcal)
D(<2500kcal)


In [77]:
sado.facts

FactList([(0, InitialFact()),
          (1, Option(foodSet='casual', option='A', calories='<1500')),
          (2, Option(foodSet='casual', option='B', calories='>=1500 & <2500')),
          (3, Option(foodSet='casual', option='C', calories='>=2500')),
          (4, Option(foodSet='moderate', option='D', calories='<2500')),
          (5,
           Option(foodSet='moderate', option='E', calories='>=2500 & <3500')),
          (6,
           Option(foodSet='moderate', option='F', calories='>=3500 & <4500')),
          (7,
           Set(setCategory='D', foodsRec=frozenlist(['Nasi Lemak', 'Nasi xde Lemak', 'Wahhh banyak lemak kau']), setCalories='<2500')),
          (8,
           Set(setCategory='E', foodsRec=frozenlist(['Nasi Goreng', 'Nasi x Goreng', 'Goreng le sendiri']), setCalories='>=2500 & <3500')),
          (9,
           Set(setCategory='F', foodsRec=frozenlist(['Nasi Mamak', 'Nasi xde Mamak', 'Teh ais sune']), setCalories='>=3500 & <4500')),
          (10, Age('20')),
        